In [1]:
import pandas as pd

# Read excel containing the original data to be masked
file = input("Please enter .xlsx file name (within same folder)")
original = pd.read_excel(file)  

Please enter .xlsx file name (within same folder)dummyoriginal.xlsx


# Masking Staff ID to 'X', where X = index+1 of list containing unique original Staff IDs

In [2]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Staff No.    50 non-null     int64         
 1   Staff        50 non-null     object        
 2   Designation  50 non-null     object        
 3   Branch       50 non-null     object        
 4   Category     50 non-null     object        
 5   Type         50 non-null     object        
 6   Course Name  50 non-null     object        
 7   Credit Days  50 non-null     float64       
 8   End Date     50 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 3.6+ KB


In [3]:
#Creating list of unique original staff IDs

unique_id = original['Staff No.'].unique().tolist()

len(unique_id) #check number of unique staff

8

In [4]:
# Note: original staff sample = 1000, but unique no of staff from 3 year consolidated training data = 996.
# This is due to 4 staff had never attended any training in past 3 years

In [5]:
def mask_id(x):
    return unique_id.index(x)+1

original['masked_id'] = original['Staff No.'].apply(mask_id)

# Masking Staff Name to 'nameX', where X = masked_id

In [6]:
def mask_name(x):
    return 'name'+str(x)

original['masked_name'] = original['masked_id'].apply(mask_name)

# Masking actual designations with general category of designation

In [7]:
# define a dictionary of actual designations and the corresponding general category = Mgr, Exc, or Ctc

job_category = {
    'BKNG SERVICES MGR' : 'Mgr',
    'CLERK/TYPIST/CASHIER' : 'Ctc',
    'BRANCH MANAGER' : 'Mgr',
    'SENIOR EXECUTIVE' : 'Exc',
    'EXECUTIVE' : 'Exc',
    'RELIEF SENIOR EXECUTIVE' : 'Exc',
    'BUSINESS MGR' : 'Mgr',
    'SR FINANCIAL EXECUTIVE (UT)' : 'Exc',
    'SENIOR EXECUTIVE (COMPLIANCE)' : 'Exc',
    'ASSISTANT MANAGER' : 'Exc',
    'SR S&M EXE' : 'Exc',
    'EXECUTIVE (CSU)' : 'Exc',
    'RELIEF ASSISTANT MANAGER' : 'Exc',
    'ASST SALES & MKTG MGR' : 'Exc',
    'DBM, BSM, DEPUTY BSM' : 'Mgr',
    'BM' : 'Mgr',
    'BIZM' : 'Mgr',
    'AM' : 'Exc'
}

In [8]:
def mask_designation(x):
    return job_category[x]

original['masked_designation'] = original['Designation'].apply(mask_designation)

# Masking actual base with BR = Branch or HO = Head Office

In [9]:
def branch_or_ho(x):
    if x == "HEAD OFFICE":
        return "HO"
    else:
        return "BR"
    
original['BR_or_HO'] = original['Branch'].apply(branch_or_ho)

# Converting all course names to course codes X, where X = index+1 of list containing unique course names

In [10]:
#Creating list of unique course names

unique_course = original['Course Name'].unique().tolist()

len(unique_course)

21

In [11]:
def assign_course_code(x):
    return unique_course.index(x)+1

original['course_code'] = original['Course Name'].apply(assign_course_code)

# Create new dataframe with masked info only

In [12]:
df_masked = original[['masked_id','masked_name','masked_designation','BR_or_HO','course_code','Category', 'Type', 'Credit Days', 'End Date' ]]

In [13]:
original = None
df_masked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   masked_id           50 non-null     int64         
 1   masked_name         50 non-null     object        
 2   masked_designation  50 non-null     object        
 3   BR_or_HO            50 non-null     object        
 4   course_code         50 non-null     int64         
 5   Category            50 non-null     object        
 6   Type                50 non-null     object        
 7   Credit Days         50 non-null     float64       
 8   End Date            50 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 3.6+ KB


In [14]:
df_masked['masked_designation'] = df_masked['masked_designation'].astype('category')
df_masked['BR_or_HO'] = df_masked['BR_or_HO'].astype('category')
df_masked['Category'] = df_masked['Category'].astype('category')
df_masked['Type'] = df_masked['Type'].astype('category')

In [15]:
df_masked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   masked_id           50 non-null     int64         
 1   masked_name         50 non-null     object        
 2   masked_designation  50 non-null     category      
 3   BR_or_HO            50 non-null     category      
 4   course_code         50 non-null     int64         
 5   Category            50 non-null     category      
 6   Type                50 non-null     category      
 7   Credit Days         50 non-null     float64       
 8   End Date            50 non-null     datetime64[ns]
dtypes: category(4), datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 3.3+ KB


# Add Columns Year & Month as extracted from End Date

In [16]:
df_masked['Year'] = df_masked['End Date'].dt.year

In [17]:
df_masked['Month'] = df_masked['End Date'].dt.month

In [18]:
df_masked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   masked_id           50 non-null     int64         
 1   masked_name         50 non-null     object        
 2   masked_designation  50 non-null     category      
 3   BR_or_HO            50 non-null     category      
 4   course_code         50 non-null     int64         
 5   Category            50 non-null     category      
 6   Type                50 non-null     category      
 7   Credit Days         50 non-null     float64       
 8   End Date            50 non-null     datetime64[ns]
 9   Year                50 non-null     int64         
 10  Month               50 non-null     int64         
dtypes: category(4), datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 4.1+ KB


# Export masked dataframe to new CSV file

In [19]:
output_file_name = input("Please enter the output file name with .csv")
df_masked.to_csv(output_file_name, index=False)

Please enter the output file name with .csvdummy-masked.csv


# The file maskedconso.csv will be used in Part 2 of the project